In [1]:
import struct
import math
import pyarrow as pa
import pyarrow.parquet as pq
import sys, os
from pathlib import Path
import dask.dataframe as dd
from time import sleep
import pandas as pd
pd.options.display.max_columns = 2000
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 10000)
from tqdm.notebook import tqdm, trange
from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 570.33 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33399,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 570.33 GiB
Comm: tcp://127.0.0.1:45271,Total threads: 8
Dashboard: http://127.0.0.1:32937/status,Memory: 35.65 GiB
Nanny: tcp://127.0.0.1:46433,


In [4]:
#client.shutdown()
#cluster.close()

In [5]:
parquet_folder="/p/gpfs1/haridev/recorder_logs/lbann_jag_32/_parquet"

In [6]:
ddf = dd.read_parquet("{}/*.parquet".format(parquet_folder), engine="pyarrow")

In [7]:
num_ranks = ddf['rank'].nunique().compute()
print(num_ranks)

128


In [8]:
ddf = ddf.sort_values('tstart')

In [9]:
posix_open = ["creat",        "creat64",      "open",         "open64"]
posix_read = ["read", "pread", "pread64", "readv"]
posix_write = ["write", "pwrite", "pwrite64","writev"]
stdio_open = ["fopen", "fopen64"]
stdio_read = ["fread"]
stdio_write = ["fwrite"]
MPI_open = ["PMPI_File_open"]
MPI_read = ["PMPI_File_iread_at", "PMPI_File_iread", "PMPI_File_iread_shared", "PMPI_File_read_all_begin",     "PMPI_File_read_all",       "PMPI_File_read_at_all",
    "PMPI_File_read_at_all_begin",  "PMPI_File_read_at",        "PMPI_File_read",
    "PMPI_File_read_ordered_begin", "PMPI_File_read_ordered",   "PMPI_File_read_shared"]
MPI_write = ["PMPI_File_iwrite_at", "PMPI_File_iwrite",             "PMPI_File_iwrite_shared","PMPI_File_write_all_begin",
    "PMPI_File_write_all",          "PMPI_File_write_at_all_begin", "PMPI_File_write_at_all",
    "PMPI_File_write_at",           "PMPI_File_write",          "PMPI_File_write_ordered_begin",
    "PMPI_File_write_ordered",      "PMPI_File_write_shared"]
HDF5_open = ["H5Fcreate",            "H5Fopen"]
HDF5_read = ["H5Dread"]
HDF5_read = ["H5Dwrite"]

In [10]:
open_funcs = posix_open
open_funcs.extend(stdio_open)
open_funcs.extend(HDF5_open)
print(open_funcs)

['creat', 'creat64', 'open', 'open64', 'fopen', 'fopen64', 'H5Fcreate', 'H5Fopen']


In [26]:
num_rows = ddf.shape[0].compute()
print(num_rows)

79709114


In [24]:
relevant_filenames = ddf[(ddf["args_1"].str.contains("/p/gpfs1/"))]['args_1'].unique().compute()
print(relevant_filenames)

0    /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext
1                                                      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy
Name: args_1, dtype: object


In [158]:
relevant_fds = ddf[(ddf["args_1"].isin(relevant_filenames))]['res'].unique().compute()
relevant_fds = relevant_fds.astype(str)
print(relevant_fds)

0    81
1    84
2    82
3    83
4    80
5    85
6    78
Name: res, dtype: object


In [159]:
reduced_ddf = ddf[~(ddf['func_id'].isin(open_funcs) & ~ddf['args_1'].isin(relevant_filenames))]

In [161]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(MPI_open) & ~reduced_ddf['args_2'].isin(relevant_filenames))]

In [162]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(posix_read) & ~reduced_ddf['args_1'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(posix_write) & ~reduced_ddf['args_1'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].str.contains("close") & ~reduced_ddf['args_1'].isin(relevant_fds))]

In [163]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(stdio_read) & ~reduced_ddf['args_4'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(stdio_write) & ~reduced_ddf['args_4'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].str.contains("fclose") & ~reduced_ddf['args_1'].isin(relevant_fds))]

In [164]:
relevant_io_funcs = ["writev" , "readv","fwrite" , 
                     "fread","pwrite" , "pread", "write" , "read",
                    "fprintf", "fopen" , "fdopen", "open", "open64", "seek", "fseek", "close", "sync", "fsync","lseek",        "lseek64",  "pread",
    "pread64",      "pwrite",       "pwrite64",     "readv",    "writev","fopen",        "fopen64",  "fclose",
    "fwrite",       "fread",        "ftell",        "fseek",    "fsync",
    "fdatasync"]

In [165]:
reduced_ddf = reduced_ddf[reduced_ddf['func_id'].isin(relevant_io_funcs)]

In [166]:
num_rows = reduced_ddf.shape[0].compute()
print(num_rows)

482977


In [167]:
filenames = reduced_ddf[(reduced_ddf["func_id"].isin(open_funcs))]['args_1'].unique().compute()
print(filenames)

0    /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext
1                                                      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy
Name: args_1, dtype: object


In [168]:
reduced_ddf = reduced_ddf[~reduced_ddf['func_id'].str.contains("MPI")]

In [169]:
reduced_ddf = reduced_ddf.compute()

In [170]:
num_rows = reduced_ddf.shape[0]
print(num_rows)

482977


In [171]:
reduced_ddf = reduced_ddf.sort_values('tstart')

In [172]:
reduced_ddf[reduced_ddf['func_id'].str.contains("read")]

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
9391,0,20.067341,20.067358,fread,0,4,%p,1,255,78,,,,,,
9394,0,20.067459,20.067472,fread,0,4,%p,1,255,78,,,,,,
9397,0,20.067593,20.067604,fread,0,4,%p,1,255,78,,,,,,
9400,0,20.067675,20.067686,fread,0,4,%p,1,255,78,,,,,,
9403,0,20.067755,20.067768,fread,0,4,%p,1,255,78,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17721,125,30.126385,30.126387,read,0,3,84,%p,8,,,,,,,
17725,125,30.126537,30.126537,read,0,3,84,%p,8,,,,,,,
17728,125,30.126570,30.126570,read,0,3,84,%p,8,,,,,,,
17819,1,30.153996,30.153999,read,0,3,84,%p,8,,,,,,,


In [173]:
reduced_ddf

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
33,2,2.364273,2.364276,fdopen,52,2,52,r,,,,,,,,
31,3,2.364273,2.364276,fdopen,52,2,52,r,,,,,,,,
33,0,2.364284,2.364288,fdopen,53,2,53,r,,,,,,,,
33,1,2.364294,2.364297,fdopen,52,2,52,r,,,,,,,,
32,4,2.364381,2.364383,fdopen,52,2,52,r,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622435,31,146.374847,146.374847,close,0,1,83,,,,,,,,,
622436,31,146.374863,146.374878,close,0,1,84,,,,,,,,,
622437,31,146.374878,146.374908,close,0,1,85,,,,,,,,,
624902,2,146.380478,146.380493,close,0,1,85,,,,,,,,,


In [174]:
relevant_filenames = reduced_ddf[(reduced_ddf["args_1"].str.contains("/p/gpfs1/"))]['args_1'].unique()
print(relevant_filenames)

['/p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext'
 '/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy']


In [183]:
closeBook = {}  # Keep track the most recent close function and its file size so a later append operation knows the most recent file size
segmentBook = {}    # segmentBook[filename] maintains all segments for filename, it is a list of list (rank, segment-id, closed)
offsetBook = {}
endOfFile = {}  # endOfFile[filename][rank] keep tracks the end of file, only the local rank can see it. When close/fsync, the value is stored in closeBook so other rank can see it.
intervals = {}

# Initialize offset book, each rank maintains its own offset book
fdSet = set([0, 1, 2])  # stdin, stderr, stdout
segmentBook["stdin"] = []
segmentBook["stderr"] = []
segmentBook["stdout"] = []
endOfFile["stdin"] = [0] * num_ranks
endOfFile["stderr"] = [0] * num_ranks
endOfFile["stdout"] = [0] * num_ranks

In [184]:
for res in relevant_fds:
    fdSet.add(res)

In [185]:
for fd in fdSet:
    offsetBook[int(fd)] = [0] * num_ranks

In [186]:
for filename in relevant_filenames:
    segmentBook[filename] = []
    endOfFile[filename] = [0] * num_ranks

In [187]:
fileMaps = []
for i in range(num_ranks):
    fileMaps.append({0: "stdin", 1: "stdout", 2: "stderr"})

In [188]:
def handle_data_operations(record, fileMap, offsetBook, endOfFile):

    def update_end_of_file(rank, fd, filename, endOfFile, offsetBook):
        if filename in endOfFile and fd in offsetBook:
            endOfFile[filename][rank] = max(endOfFile[filename][rank], offsetBook[fd][rank])
        elif filename not in endOfFile:
            endOfFile[filename] = [0] * len(offsetBook[0])
            endOfFile[filename][rank] = offsetBook[fd][rank]
        else:
            print("Not possible: ", rank, fd, filename)
            endOfFile[filename][rank] = 0

    func = record['func_id']
    rank = record['rank']

    filename, offset, count = "", -1, -1

    # Ignore the functions that may confuse later conditions test
    if "readlink" in func or "dir" in func:
        return filename, offset, count

    if "writev" in func or "readv" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fwrite" in func or "fread" in func:
        fd, size, count = int(record['args_4']), int(record['args_2']), int(record['args_3'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset, count = offsetBook[fd][rank], size*count
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "pwrite" in func or "pread" in func:
        fd, count, offset = int(record['args_1']), int(record['args_3']), int(record['args_4'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "write" in func or "read" in func:
        fd, count = int(record['args_1']), int(record['args_3'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fprintf" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)

    return filename, offset, count


def handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile):

    def get_latest_offset(filename, rank, closeBook, endOfFile):
        # Every filename should be in endOfFile becuase we initialized it at the begining
        if filename not in closeBook and filename not in endOfFile:
            print("Encounter an unknown filename "+filename+". Should abort!")
            return 0
        if filename in closeBook:
            return max(endOfFile[filename][rank], closeBook[filename])
        else:
            return endOfFile[filename][rank]

    def create_new_segment(filename, rank, segmentBook):
        newSegmentID = 0
        if filename in segmentBook and len(segmentBook[filename]) > 0:
            newSegmentID = 1+segmentBook[filename][-1][1]
        if filename not in segmentBook:
            segmentBook[filename] = []
        segmentBook[filename].append([rank, newSegmentID, False])

    rank, func = record['rank'], record['func_id']

    # Ignore directory related operations
    if "dir" in func:
        return

    if "fopen" in func or "fdopen" in func:
        fd = record['res']
        if "fdopen" in func:
            oldFd = int(record['args_1'])
            if oldFd not in fileMap:        # openning some file that we do not know?
                return
            else:
                filename = fileMap[oldFd]
        else:
            filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = record['args_2']
        if 'a' in openMode:
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "open" in func:
        fd = record['res']
        filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = int( record['args_2'] )
        if openMode == 2:  # TODO need  a better way to test for O_APPEND
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "seek" in func:
        fd, offset, whence = int(record['args_1']), int(record['args_2']), int(record['args_3'])
        if fd not in fileMap: return
        filename = fileMap[fd]

        if whence == 0:     # SEEK_SET
            offsetBook[fd][rank] = offset
        elif whence == 1:   # SEEK_CUR
            offsetBook[fd][rank] += offset
        elif whence == 2:   # SEEK_END
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)

    elif "close" in func or "sync" in func:
        fd = int(record['args_1'])
        if(fd not in fileMap): return
        filename = fileMap[fd]
        if "close" in func: del fileMap[fd]

        closeBook[filename] = endOfFile[filename][rank]

        # 1. Close all segments on the local process for this file
        for i in range(len(segmentBook[filename])):
            if segmentBook[filename][i][0] == rank:
                segmentBook[filename][i][2] = True
        # 2. And starts a new segment for all other processes have the same file opened
        # Skip this step for session semantics
        visitedRanks = set()
        tmpSegments = []
        # [::-1] check the most recent unclosed segment to get the largest segmentId
        for segment in segmentBook[filename][::-1]:
            if segment[0] in visitedRanks:
                continue
            if segment[0] != rank and not segment[2]:
                tmpSegments.append([segment[0], 1+segment[1], False])
                visitedRanks.add(segment[0])
        segmentBook[filename] = segmentBook[filename] + tmpSegments


def ignore_files(filename):
    if not filename or filename == "":
        return True
    ignore_prefixes = ["/sys/", "/dev", "/proc", "/etc/", "stdout", "stderr", "stdin"]
    for prefix in ignore_prefixes:
        if filename.startswith(prefix):
            return True
    if "pipe:" in filename:
        return True

    return False

def ignore_funcs(func):
    ignore = ["MPI", "H5", "writev"]
    for f in ignore:
        if f in func:
            return True
    return False

In [195]:
#bar = trange(num_rows)
for index, record in tqdm(reduced_ddf.iterrows(), total=num_rows):
    #bar.set_description(f'Working on "{index}"')
    #print(f'Working on {index}', end='\r')
    
    rank = record['rank']
    #print(rank)
    fileMap = fileMaps[rank]

    func = record['func_id']

    #print(f'Handle Meta on {index}', end='\r')
    handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile)
    
    #print(f'Handle Data on {index}', end='\r')
    filename, offset, count = handle_data_operations(record, fileMap, offsetBook, endOfFile)
    
    #if filename:
    #    print(filename, offset, count)
    if not ignore_files(filename):
        isRead = "read" in func
        if filename not in intervals:
            intervals[filename] = []

        # segments[0] is the local segment, the others are remote segments
        segments = []
        # 1. Add local segment
        # Find the most recent unclosed local segment
        #print(f'Local Segment on {index}', end='\r')
        for segment in segmentBook[filename][::-1]:
            if segment[0] == rank and not segment[2]:
                segments.append(segment[1])

        # 2. Add all remote segmentsv
        #print(f'Remote Segment on {index}', end='\r')
        for segment in segmentBook[filename]:
            if segment[0] != rank and not segment[2]:
                segments.append(segment[1])
        intervals[filename].append( [rank, record.tstart, record.tend, offset, count, isRead, segments] )

  0%|          | 0/482977 [00:00<?, ?it/s]

In [197]:
def function_patterns(all_intervals):
    # 1,2,3 - consecutive
    # 1,3,9 - sequential
    # 1,3,2 - random
    x = {'consecutive':0, 'sequential':0, 'random':0}
    for filename in all_intervals.keys():
        if ignore_files(filename): continue
        intervals = sorted(all_intervals[filename], key=lambda x: x[1])   # sort by tstart
        '''
        This code consider each rank separately
        lastOffsets = [0] * reader.globalMetadata.numRanks
        for interval in intervals:
            rank, offset, count = interval[0], interval[3], interval[4]
            lastOffset = lastOffsets[rank]
            if (offset + count) == lastOffset:
                x['consecutive'] += 1
            elif (offset + count) > lastOffset:
                x['sequential'] += 1
            else:
                #print filename, interval
                x['random'] += 1
            lastOffsets[rank] = offset + count
        '''
        for i in range(len(intervals)-1):
            i1, i2 = intervals[i], intervals[i+1]
            offset1, count1 = i1[3], i1[4]
            offset2, count2 = i2[3], i2[4]

            if (offset1 + count1) == offset2:
                x['consecutive'] += 1
            elif (offset1 + count1) < offset2:
                x['sequential'] += 1
            else:
                x['random'] += 1
        total = x['consecutive'] + x['sequential'] + x['random']
    print("consecutive:",  x['consecutive'] )
    print("sequential:",  x['sequential'] )
    print("random:",  x['random'])

In [198]:
function_patterns(intervals)

consecutive: 5
sequential: 0
random: 1273
